# Welcome to Exkaldi

In this section, we will make a HCLG decode graph.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Prepare lexicons (generated in 3_prepare_lexicons).

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

Like the traing of HMM and decision tree, we also have high-level API to make HCLG graph. But firstly we introduce how to build HCLG graph with exkaldi toolkit in detail.

### Make HCLG in detail

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_delta", "graph")

exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)

Then compose __L.fst__ and __G.fst__ to __LG.fst__. 

__L.fst__ file has been generated before (in 3_prepare_lexicons), use it directly.  
__G.fst__ file has been generated before (in 04_train_and_query_language_model), use it directly.

In [ ]:
Lfile = os.path.join(dataDir, "exp", "L_disambig.fst")
Gfile = os.path.join(dataDir, "exp", "G.fst")
LGfile = os.path.join(outDir, "LG.fst")

exkaldi.decode.graph.compose_LG(Lfile, Gfile, outFile=LGfile)

Then compose __LG__ and context information to __CLG.fst__. 

__tree__ will be used here. and __ilabel__ info will also be generated in this step.

In [ ]:
treeFile = os.path.join(dataDir, "exp", "train_delta", "tree")
CLGfile = os.path.join(outDir, "CLG.fst")

_, ilabelFile = exkaldi.decode.graph.compose_CLG(lexicons, treeFile, LGfile, outFile=CLGfile)

ilabelFile

Last, compose all infos to final __HCLG.fst__ graph.

In [ ]:
hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")
HCLGFile = os.path.join(outDir, "HCLG.fst")

exkaldi.decode.graph.compose_HCLG(hmmFile, treeFile, CLGfile, ilabelFile, outFile=HCLGFile)

### Make HCLG with high-level API

In [ ]:
os.remove(LGfile)
os.remove(CLGfile)
os.remove(ilabelFile)
os.remove(HCLGFile)

In [ ]:
exkaldi.decode.graph.make_graph(lexicons, hmmFile, treeFile, tempDir=outDir, useLfile=Lfile, useGfile=Gfile)

Here we specified "useLfile" and "useGfile" to use the L and G fst file generated before. If you don't want this, specify "useLfile" and "useGfile" None, and other parameters, such as "useDisambigLexicons" and so on.